# Ultimate Bending Capacity

This example demonstrates how to calculate ultimate bending capacities using ``concreteproperties``. We start by importing the necessary modules.


In [ ]:
import numpy as np
from sectionproperties.pre.library import rectangular_section, triangular_section

import concreteproperties.stress_strain_profile as ssp
from concreteproperties import (
    Concrete,
    ConcreteSection,
    SteelBar,
    add_bar_rectangular_array,
)

## Assign Materials
The materials used in this example will be 50 MPa concrete with 500 MPa steel, specified in accordance with AS 3600:2018.

In [ ]:
concrete = Concrete(
    name="50 MPa Concrete",
    density=2.4e-6,
    stress_strain_profile=ssp.ConcreteLinear(elastic_modulus=34.8e3),
    ultimate_stress_strain_profile=ssp.RectangularStressBlock(
        compressive_strength=50,
        alpha=0.775,
        gamma=0.845,
        ultimate_strain=0.003,
    ),
    flexural_tensile_strength=4.2,
    colour="lightgrey",
)

steel = SteelBar(
    name="500 MPa Steel",
    density=7.85e-6,
    stress_strain_profile=ssp.SteelElasticPlastic(
        yield_strength=500,
        elastic_modulus=200e3,
        fracture_strain=0.05,
    ),
    colour="grey",
)

## Create Geometry and Concrete Section
The section being analysed in this example is a 1200D x 900W x 150THK hollow box beam with 50 mm chamfers. The reinforcement detailed is 9N20 top bars and 9N28 bottom bars with 6N16 side face. The bars are cast central to the walls. The geometry is generated using the *sectionproperties* primitive sections library and geometry manipulation.

In [ ]:
# construct box by subtracting an inner rectangle from an outer rectangle
outer = rectangular_section(d=1200, b=900, material=concrete)
inner = rectangular_section(d=900, b=600).align_center(align_to=outer)
box = outer - inner

# generate four chamfers
chamfer1 = (
    triangular_section(b=50, h=50, material=concrete)
    .align_to(other=inner, on="left", inner=True)
    .align_to(other=inner, on="bottom", inner=True)
)
chamfer2 = chamfer1.mirror_section(axis="y", mirror_point=(450, 600))
chamfer3 = chamfer1.mirror_section(axis="x", mirror_point=(450, 600))
chamfer4 = chamfer2.mirror_section(axis="x", mirror_point=(450, 600))

# add chamfers to box
geom = box + chamfer1 + chamfer2 + chamfer3 + chamfer4

# add bottom bars
geom = add_bar_rectangular_array(
    geometry=geom,
    area=620,
    material=steel,
    n_x=9,
    x_s=750 / 8,
    anchor=(75, 75),
)

# add top bars
geom = add_bar_rectangular_array(
    geometry=geom,
    area=310,
    material=steel,
    n_x=9,
    x_s=750 / 8,
    anchor=(75, 1125),
)

# add side bars
geom = add_bar_rectangular_array(
    geometry=geom,
    area=200,
    material=steel,
    n_x=2,
    x_s=750,
    n_y=6,
    y_s=150,
    anchor=(75, 225),
)

conc_sec = ConcreteSection(geom)
conc_sec.plot_section()

## Calclulate Ultimate Properties ($N = 0$)
In this example we’ll calculate the ultimate bending capacity for sagging ($\theta = 0$), hogging ($\theta = \pi$) and weak axis bending ($\theta = \pi / 2$). To do this we call the ``ultimate_bending_capacity()`` method:

In [ ]:
sag_res = conc_sec.ultimate_bending_capacity()
hog_res = conc_sec.ultimate_bending_capacity(theta=np.pi)
weak_res = conc_sec.ultimate_bending_capacity(theta=np.pi / 2)

## Displaying the Results  ($N = 0$)
We can print the ultimate bending results to the terminal by using the ``print_results()`` method.

In [ ]:
sag_res.print_results()

In [ ]:
hog_res.print_results()

In [ ]:
weak_res.print_results()

We can also extract specific results from the ``UltimateBendingResults`` objects. Note that ``m_xy`` refers to the resultant bending moment, so will always be positive.

In [ ]:
print(f"M_x+ = {sag_res.m_xy / 1e6:.2f} kN.m")
print(f"M_x- = {hog_res.m_xy / 1e6:.2f} kN.m")
print(f"M_y = {weak_res.m_xy / 1e6:.2f} kN.m")

## Calclulate Ultimate Properties ($N = 5000$ kN)
Say our reinforced concrete cross-section has an axial compression of 5000 kN. We can also calculate the ultimate bending capacities given this axial load.

In [ ]:
n = 5000e3
sag_axial_res = conc_sec.ultimate_bending_capacity(n=n)
hog_axial_res = conc_sec.ultimate_bending_capacity(theta=np.pi, n=n)
weak_axial_res = conc_sec.ultimate_bending_capacity(theta=np.pi / 2, n=n)

## Displaying the Results  ($N = 5000$ kN)


In [ ]:
msg_sag = f"M_x+ = {sag_axial_res.m_xy / 1e6:.1f} kN.m "
msg_sag += f"with N = {sag_axial_res.n / 1e3:.0f} kN"
msg_hog = f"M_x- = {hog_axial_res.m_xy / 1e6:.1f} kN.m "
msg_hog += f"with N = {hog_axial_res.n / 1e3:.0f} kN"
msg_weak = f"M_y = {weak_axial_res.m_xy / 1e6:.1f} kN.m "
msg_weak += f"with N = {weak_axial_res.n / 1e3:.0f} kN"

print(msg_sag)
print(msg_hog)
print(msg_weak)